<a href="https://colab.research.google.com/github/vence-andersen/M5-Forecasting-Accuracy/blob/main/Ensemble_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Removables**

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"grahamfernando","key":"cffa546ccec7808cfc8cec9944ed384a"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download -c m5-forecasting-accuracy

 57% 9.00M/15.8M [00:00<00:00, 20.5MB/s]
100% 15.8M/15.8M [00:00<00:00, 29.4MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 98.5MB/s]
 58% 9.00M/15.5M [00:00<00:00, 13.1MB/s]
100% 15.5M/15.5M [00:00<00:00, 16.9MB/s]
 63% 9.00M/14.2M [00:00<00:00, 19.0MB/s]
100% 14.2M/14.2M [00:00<00:00, 26.4MB/s]
  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 132MB/s]


In [7]:
!unzip sales_train_evaluation.csv.zip
!unzip sales_train_validation.csv.zip
!unzip sample_submission.csv.zip
!unzip sell_prices.csv.zip

Archive:  sales_train_evaluation.csv.zip
  inflating: sales_train_evaluation.csv  
Archive:  sales_train_validation.csv.zip
  inflating: sales_train_validation.csv  
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  sell_prices.csv.zip
  inflating: sell_prices.csv         


In [1]:
! pip install downcast
! pip install catboost

     |████████████████████████████████| 65.8MB 64kB/s 


# **Imports**

In [32]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import pickle
import dill
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error as mse
from typing import Union
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/new.pkl')
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_28,lag_30,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_median_5,rolling_median_7,rolling_median_28,rolling_median_56
30490000,14370,1437,3,1,0,0,1001,2,11339,7,10,2013,-1,-1,-1,-1,0,0,0,8.257812,1.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
30490001,14380,1438,3,1,0,0,1001,0,11339,7,10,2013,-1,-1,-1,-1,0,0,0,3.970703,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.drop(['rolling_median_5','rolling_median_7','rolling_median_28','rolling_median_56'],axis=1, inplace=True)

In [ ]:
# Now we are splitting our data into, Train, Test, Cross Validate.
# Being a time series model, we are splitting the data based on time.

# Records till day 1914 will be used for training the model.
X_train = data[data['day']<1800]

# Records after day 1942 will used for final test
X_test = data[(data['day']>=1800)]

y_train = X_train['demand']
y_test = X_test['demand']

# We are drpping the features which are not required.
X_train.drop(['demand'],axis = 1,inplace = True)
X_test.drop(['demand'],axis = 1,inplace = True)

print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(24361510, 27) (24361510,)
(5183300, 27) (5183300,)


In [ ]:
X_train_D1 = X_train[X_train['day']<=1400]
X_train_D2 = X_train[X_train['day']>1400]

y_train_D1 = y_train[y_train.index <= X_train_D1.index[-1]].reset_index(drop=True)
y_train_D2 = y_train[y_train.index > X_train_D1.index[-1]].reset_index(drop=True)

X_train_D1.reset_index(drop=True, inplace=True)
X_train_D2.reset_index(drop=True, inplace=True)

In [ ]:
def sampler(max_val):

    siz = np.int(max_val * 0.70)
    return np.round(np.random.uniform(0,max_val-1, size=siz)).astype(int).tolist()

In [ ]:
k1_train = X_train_D1.iloc[sampler(X_train_D1.shape[0])].sort_index()
k1_test = y_train_D1[k1_train.index.tolist()]

k2_train = X_train_D1.iloc[sampler(X_train_D1.shape[0])].sort_index()
k2_test = y_train_D1[k2_train.index.tolist()]

k3_train = X_train_D1.iloc[sampler(X_train_D1.shape[0])].sort_index()
k3_test = y_train_D1[k3_train.index.tolist()]

k4_train = X_train_D1.iloc[sampler(X_train_D1.shape[0])].sort_index()
k4_test = y_train_D1[k4_train.index.tolist()]

k5_train = X_train_D1.iloc[sampler(X_train_D1.shape[0])].sort_index()
k5_test = y_train_D1[k5_train.index.tolist()]

In [ ]:
params = {'alpha' : [0.001,0.01,0.1,1,10,0.05,0.5]}

lasso = Lasso()
clf = GridSearchCV(estimator=lasso, param_grid=params,scoring='neg_mean_squared_error', cv=3,n_jobs=-1, return_train_score=True, verbose=2)
clf.fit(k1_train, k1_test)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  4.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 0.05, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,125.738910,89.120867,0.366682,0.093678,0.001,{'alpha': 0.001},-8.890647,-6.584685,-5.549262,-7.008198,1.396599,6,-6.008701,-6.614247,-7.117569,-6.580172,0.453334
1,31.778010,8.568656,0.488238,0.004366,0.01,{'alpha': 0.01},-7.834643,-6.588242,-5.548908,-6.657264,0.934423,1,-6.010812,-6.615561,-7.119207,-6.581860,0.453127
2,15.063405,1.511486,0.452762,0.058067,0.1,{'alpha': 0.1},-7.817636,-6.603689,-5.560840,-6.660722,0.922215,3,-6.015190,-6.620996,-7.125393,-6.587193,0.453868
3,12.600375,0.889002,0.463780,0.068755,1,{'alpha': 1},-7.688156,-6.822164,-5.720143,-6.743488,0.805362,5,-6.118202,-6.717062,-7.228772,-6.688012,0.453854
4,3.977889,0.842218,0.388808,0.054074,10,{'alpha': 10},-14.365685,-14.036357,-13.362334,-13.921459,0.417596,7,-13.698459,-13.862901,-14.199687,-13.920349,0.208619
5,17.392705,4.912009,0.502443,0.033038,0.05,{'alpha': 0.05},-7.828810,-6.595367,-5.556068,-6.660082,0.928971,2,-6.013965,-6.618291,-7.123939,-6.585398,0.453742
6,12.112609,1.990765,0.328401,0.108668,0.5,{'alpha': 0.5},-7.737507,-6.687425,-5.614484,-6.679805,0.866737,4,-6.040975,-6.647594,-7.154391,-6.614320,0.455158


In [ ]:
clf.best_params_

{'alpha': 0.01}

In [ ]:
model_1 = Lasso(alpha=0.01)
model_1.fit(k1_train, k1_test)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
params = {'alpha' : [0.001,0.01,0.1,1,10,0.05,0.5],
          'l1_ratio' : [0.01,0.1,1]}

elastic_net = ElasticNet(max_iter=250)
clf = GridSearchCV(estimator=elastic_net, param_grid=params, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, return_train_score=True)
clf.fit(k2_train, k2_test)

GridSearchCV(cv=3, error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=250, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 0.05, 0.5],
                         'l1_ratio': [0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
pd.DataFrame(clf.cv_results_)[['param_alpha','param_l1_ratio','mean_test_score','std_test_score','mean_train_score','std_train_score']]

,param_alpha,param_l1_ratio,mean_test_score,std_test_score,mean_train_score,std_train_score
0,0.001,0.01,-6.783339,1.148269,-6.549112,0.456022
1,0.001,0.1,-6.780553,1.144477,-6.549129,0.456033
2,0.001,1,-6.746563,1.098752,-6.549222,0.456024
3,0.01,0.01,-6.761373,1.118671,-6.549169,0.456042
4,0.01,0.1,-6.733826,1.081732,-6.549260,0.456014
5,0.01,1,-6.631170,0.943574,-6.550869,0.455858
6,0.1,0.01,-6.674251,1.002016,-6.549727,0.456050
7,0.1,0.1,-6.631055,0.942717,-6.551353,0.456032
8,0.1,1,-6.634286,0.930460,-6.556403,0.456654
9,1,0.01,-6.633117,0.934295,-6.555253,0.456452


In [ ]:
clf.best_params_

{'alpha': 0.1, 'l1_ratio': 0.1}

In [ ]:
model_2 = ElasticNet(alpha=0.1,l1_ratio=0.1,max_iter=250)
model_2.fit(k2_train, k2_test)

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.1,
           max_iter=250, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
model_3 = LinearRegression(n_jobs=-1)
model_3.fit(k3_train, k3_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [ ]:
a = pd.DataFrame()
for i in range(7):    
    params = {'learning_rate' : np.round(np.random.rand(1)/10,3).tolist(),
            'min_data_in_leaf' : np.random.randint(30,113, size=1).tolist()}

    cat_boost = CatBoostRegressor(logging_level="Silent", iterations=130)
    clf = GridSearchCV(estimator=cat_boost, param_grid=params, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, return_train_score=True)
    clf.fit(k4_train, k4_test)
    b = pd.DataFrame(clf.cv_results_)
    a = pd.concat([a,b])

In [ ]:
a['diff'] = a['mean_test_score'] - a['mean_train_score']
a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_min_data_in_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,diff
0,207.468141,1.687388,7.824273,0.553943,0.09,74,"{'learning_rate': 0.09, 'min_data_in_leaf': 74}",-7.379380,-6.462665,-6.009775,-6.617273,0.569726,1,-5.421570,-5.713077,-5.938458,-5.691035,0.211594,-0.926238
0,202.502698,1.565536,7.910024,1.140081,0.019,108,"{'learning_rate': 0.019, 'min_data_in_leaf': 108}",-7.427849,-6.807751,-6.227346,-6.820982,0.490193,1,-6.109435,-6.510635,-6.744936,-6.455002,0.262408,-0.365980
0,204.804547,5.129653,8.428418,1.250131,0.099,92,"{'learning_rate': 0.099, 'min_data_in_leaf': 92}",-7.359930,-6.484319,-6.001131,-6.615127,0.562386,1,-5.396566,-5.673982,-5.881579,-5.650709,0.198688,-0.964418
0,198.191542,2.016174,7.639589,0.999844,0.013,62,"{'learning_rate': 0.013, 'min_data_in_leaf': 62}",-7.728771,-7.230054,-6.647829,-7.202218,0.441731,1,-6.600881,-6.996273,-7.236005,-6.944386,0.261871,-0.257831
0,205.032764,4.429652,8.366679,1.152466,0.095,63,"{'learning_rate': 0.095, 'min_data_in_leaf': 63}",-7.375966,-6.441693,-5.975907,-6.597855,0.582140,1,-5.412212,-5.691887,-5.905400,-5.669833,0.201946,-0.928022
0,196.218116,1.444107,7.505073,0.695449,0.007,33,"{'learning_rate': 0.007, 'min_data_in_leaf': 33}",-8.834619,-8.500979,-7.981819,-8.439139,0.350889,1,-7.990521,-8.331538,-8.539963,-8.287341,0.226476,-0.151799
0,211.047831,2.886530,8.068245,1.519539,0.096,56,"{'learning_rate': 0.096, 'min_data_in_leaf': 56}",-7.391535,-6.458575,-5.985974,-6.612028,0.583987,1,-5.399660,-5.682874,-5.891194,-5.657909,0.201443,-0.954119


In [ ]:
model_4 = CatBoostRegressor(learning_rate =  0.019, min_data_in_leaf =  108, logging_level="Silent", iterations=130)
model_4.fit(k4_train, k4_test)

In [ ]:
a = pd.DataFrame()
for i in range(7):    
    params = {'learning_rate' : np.round(np.random.rand(1)/10,3).tolist(),
              'min_data_in_leaf' : np.random.randint(30,113, size=1).tolist(),
              'num_leaves' : np.random.randint(30,120, size=1).tolist()}

    lgbm = LGBMRegressor()
    clf = GridSearchCV(estimator=lgbm, param_grid=params, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, return_train_score=True)
    clf.fit(k5_train, k5_test)
    b = pd.DataFrame(clf.cv_results_)
    a = pd.concat([a,b])

In [ ]:
a['diff'] = a['mean_test_score'] - a['mean_train_score']
a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_min_data_in_leaf,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,diff
0,189.623296,3.354854,24.454322,1.091955,0.026,75,118,"{'learning_rate': 0.026, 'min_data_in_leaf': 7...",-7.825666,-6.973231,-6.256847,-7.018581,0.641270,1,-5.620329,-5.982386,-6.210754,-5.937823,0.243091,-1.080758
0,203.156685,6.510428,24.678571,0.952774,0.017,104,105,"{'learning_rate': 0.017, 'min_data_in_leaf': 1...",-7.984280,-7.332743,-6.422263,-7.246429,0.640605,1,-6.200640,-6.603033,-6.863651,-6.555774,0.272728,-0.690654
0,195.459994,4.318202,25.438684,0.617221,0.024,65,117,"{'learning_rate': 0.024, 'min_data_in_leaf': 6...",-7.848210,-7.022230,-6.349479,-7.073306,0.612920,1,-5.665672,-6.029238,-6.252568,-5.982492,0.241868,-1.090814
0,159.530903,4.100867,18.977489,0.707780,0.024,32,60,"{'learning_rate': 0.024, 'min_data_in_leaf': 3...",-7.892070,-7.071898,-6.352282,-7.105417,0.629062,1,-5.750866,-6.085426,-6.324580,-6.053624,0.235295,-1.051793
0,132.587227,4.993513,15.063794,0.087317,0.092,77,34,"{'learning_rate': 0.092, 'min_data_in_leaf': 7...",-7.925334,-6.854090,-6.385951,-7.055125,0.644327,1,-5.334187,-5.626790,-5.873110,-5.611362,0.220284,-1.443763
0,157.793305,5.395960,18.996548,1.578407,0.071,70,75,"{'learning_rate': 0.071, 'min_data_in_leaf': 7...",-8.017177,-6.843177,-6.386655,-7.082336,0.686803,1,-5.109191,-5.378968,-5.585199,-5.357786,0.194906,-1.724550
0,142.667153,4.412197,17.161679,0.881772,0.091,63,55,"{'learning_rate': 0.091, 'min_data_in_leaf': 6...",-7.993168,-6.837662,-6.347488,-7.059439,0.689906,1,-5.110973,-5.350050,-5.564041,-5.341688,0.185059,-1.717751


In [ ]:
model_5 = LGBMRegressor(learning_rate =  0.017, min_data_in_leaf =  104, num_leaves = 105)
model_5.fit(k5_train, k5_test)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.017, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=104, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=105, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [ ]:
filename = 'globalsave.pkl'
dill.dump_session(filename)

In [ ]:
!cp globalsave.pkl /content/drive/MyDrive

In [8]:
dill.load_session('/content/drive/MyDrive/globalsave.pkl')

In [ ]:
for i in range(1,6):
    pickle_out = open("model_"+str(i)+".pickle","wb")
    pickle.dump(globals()['model_'+str(i)], pickle_out)
    pickle_out.close()

In [ ]:
!cp model_1.pickle /content/drive/MyDrive
!cp model_2.pickle /content/drive/MyDrive
!cp model_3.pickle /content/drive/MyDrive
!cp model_4.pickle /content/drive/MyDrive
!cp model_5.pickle /content/drive/MyDrive

In [ ]:
model_1 = pickle.load(open('model_1.pickle','rb'))
model_2 = pickle.load(open('model_2.pickle','rb'))
model_3 = pickle.load(open('model_3.pickle','rb'))
model_4 = pickle.load(open('model_4.pickle','rb'))
model_5 = pickle.load(open('model_5.pickle','rb'))

In [9]:
k1_pred = pd.Series(model_1.predict(X_train_D2))
k2_pred = pd.Series(model_2.predict(X_train_D2))
k3_pred = pd.Series(model_3.predict(X_train_D2))
k4_pred = pd.Series(model_4.predict(X_train_D2))
k5_pred = pd.Series(model_5.predict(X_train_D2))

In [10]:
X_train_D2_pred = pd.concat([pd.DataFrame(k1_pred, columns=['k1_pred']), pd.DataFrame(k2_pred,columns=['k2_pred']),\
           pd.DataFrame(k3_pred,columns=['k3_pred']),pd.DataFrame(k4_pred,columns=['k4_pred']),\
           pd.DataFrame(k5_pred,columns=['k5_pred'])],axis=1)

X_train_D2_pred.head(2)

,k1_pred,k2_pred,k3_pred,k4_pred,k5_pred
0,0.847898,0.833735,0.786133,0.605631,0.758412
1,0.285196,0.278305,0.214355,0.446354,0.407913


In [11]:
test_pred_1 = pd.Series(model_1.predict(X_test))
test_pred_2 = pd.Series(model_2.predict(X_test))
test_pred_3 = pd.Series(model_3.predict(X_test))
test_pred_4 = pd.Series(model_4.predict(X_test))
test_pred_5 = pd.Series(model_5.predict(X_test))

In [12]:
predictions = pd.concat([pd.DataFrame(test_pred_1, columns=['test_pred_1']), pd.DataFrame(test_pred_2,columns=['test_pred_2']),\
           pd.DataFrame(test_pred_3,columns=['test_pred_3']),pd.DataFrame(test_pred_4,columns=['test_pred_4']),\
           pd.DataFrame(test_pred_5,columns=['test_pred_5'])],axis=1)

predictions.head(2)

,test_pred_1,test_pred_2,test_pred_3,test_pred_4,test_pred_5
0,0.536272,0.537838,0.619873,0.478783,0.558100
1,1.215574,1.209943,1.297119,0.934893,1.021989


In [16]:
for i in range(15):
    
    lr = np.round(np.random.rand()/10,3)
    num_leaves = np.random.randint(30,150)
    min_data_in_leaf = np.random.randint(50,150)
    
    lgb = LGBMRegressor(learning_rate=lr ,num_leaves=num_leaves ,min_data_in_leaf=min_data_in_leaf)
    lgb.fit(X_train_D2_pred, y_train_D2)
    
    train_pred = lgb.predict(X_train_D2_pred)

    train_m_s_e = mse(y_train_D2,train_pred)
    # After training the model, we are trying to predict the model on X_val to check it's accuracy
    y_pred = lgb.predict(predictions)
    
    # Now that it has predicted the values for X_val, we are calculating it's rmse
    m_s_e = mse(y_test,y_pred)

    print(f"For learning rate {lr}, num_leaves {num_leaves} and min_data_in_leaf {min_data_in_leaf}, the train MSE is {train_m_s_e} and valid MSE is {m_s_e} diff is {m_s_e - train_m_s_e}")
    print('*'*80)

For learning rate 0.066, num_leaves 140 and min_data_in_leaf 52, the train MSE is 6.346031512687311 and valid MSE is 6.538802100752078 diff is 0.192770588064767
********************************************************************************
For learning rate 0.003, num_leaves 72 and min_data_in_leaf 91, the train MSE is 9.726929508281916 and valid MSE is 8.705906834632353 diff is -1.0210226736495631
********************************************************************************
For learning rate 0.047, num_leaves 116 and min_data_in_leaf 132, the train MSE is 6.361618953779319 and valid MSE is 6.523922152936729 diff is 0.1623031991574102
********************************************************************************
For learning rate 0.081, num_leaves 65 and min_data_in_leaf 87, the train MSE is 6.358918970665034 and valid MSE is 6.536539029399947 diff is 0.17762005873491304
********************************************************************************
For learning rate 0.077, num

In [62]:
meta_model = LGBMRegressor(learning_rate=0.074 ,num_leaves=135 ,min_data_in_leaf=85)
meta_model.fit(X_train_D2_pred, y_train_D2)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.074, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=85,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=135,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [71]:
predictions_cbr = predictions
predictions_cbr.columns = ['k1_pred','k2_pred','k3_pred','k4_pred','k5_pred']
predictions_cbr

,k1_pred,k2_pred,k3_pred,k4_pred,k5_pred
0,0.536272,0.537838,0.619873,0.478783,0.558100
1,1.215574,1.209943,1.297119,0.934893,1.021989
2,0.980738,1.006427,1.052734,0.732418,0.905047
3,2.242874,2.181285,2.316162,1.650242,1.841847
4,1.563980,1.551696,1.644287,1.175259,1.257319
...,...,...,...,...,...
5183295,0.949158,0.975781,1.009766,0.761162,0.862908
5183296,0.346672,0.321799,0.387695,0.440660,0.439751
5183297,1.184471,1.172040,1.246582,0.785601,0.890500
5183298,1.110920,1.073041,1.143066,0.910342,1.025053


In [74]:
for i in range(10):
    
    lr = np.round(np.random.rand()/10,3)
    # num_leaves = np.random.randint(30,150)
    min_data_in_leaf = np.random.randint(50,150)

    cbr = CatBoostRegressor(learning_rate=lr, min_data_in_leaf=min_data_in_leaf, logging_level="Silent", iterations=130)
    cbr.fit(X_train_D2_pred, y_train_D2)

    train_pred = cbr.predict(X_train_D2_pred)

    train_m_s_e = mse(y_train_D2,train_pred)
    # After training the model, we are trying to predict the model on X_val to check it's accuracy
    y_pred = cbr.predict(predictions_cbr)

    # Now that it has predicted the values for X_val, we are calculating it's rmse
    m_s_e = mse(y_test,y_pred)

    print(f"For learning rate {lr}, min_data_in_leaf {min_data_in_leaf}, the train MSE is {train_m_s_e} and valid MSE is {m_s_e} diff is {m_s_e - train_m_s_e}")
    print('*'*80)

For learning rate 0.038, min_data_in_leaf 146, the train MSE is 6.203251585588742 and valid MSE is 6.4074725023075745 diff is 0.2042209167188327
********************************************************************************
For learning rate 0.026, min_data_in_leaf 62, the train MSE is 6.217091860325036 and valid MSE is 6.363763826231865 diff is 0.14667196590682874
********************************************************************************
For learning rate 0.001, min_data_in_leaf 83, the train MSE is 11.082284008502336 and valid MSE is 9.8829209468545 diff is -1.199363061647837
********************************************************************************
For learning rate 0.028, min_data_in_leaf 94, the train MSE is 6.2124292661499965 and valid MSE is 6.374732647219512 diff is 0.16230338106951514
********************************************************************************
For learning rate 0.024, min_data_in_leaf 127, the train MSE is 6.22366563872105 and valid MSE is 6.

In [86]:
meta_model_cbr = CatBoostRegressor(learning_rate=0.099, min_data_in_leaf = 107, iterations=100, logging_level="Silent")
meta_model_cbr.fit(X_train_D2_pred, y_train_D2)

In [24]:
val = X_test[(X_test['day']>1913) & (X_test['day']<1942)]
tst = X_test[X_test['day']>1941]

In [25]:
final_1_val = model_1.predict(val)
final_1_tst = model_1.predict(tst)

final_2_val = model_2.predict(val)
final_2_tst = model_2.predict(tst)

final_3_val = model_3.predict(val)
final_3_tst = model_3.predict(tst)

final_4_val = model_4.predict(val)
final_4_tst = model_4.predict(tst)

final_5_val = model_5.predict(val)
final_5_tst = model_5.predict(tst)

In [26]:
final_val_preds = pd.concat([pd.DataFrame(final_1_val, columns=['final_1_val']), pd.DataFrame(final_2_val,columns=['final_2_val']),\
           pd.DataFrame(final_3_val,columns=['final_3_val']),pd.DataFrame(final_4_val,columns=['final_4_val']),\
           pd.DataFrame(final_5_val,columns=['final_5_val'])],axis=1)

final_val_preds.head(2)

,final_1_val,final_2_val,final_3_val,final_4_val,final_5_val
0,0.976135,0.984558,0.918457,0.811426,0.884323
1,0.692354,0.687013,0.646240,0.531709,0.609000


In [27]:
final_tst_preds = pd.concat([pd.DataFrame(final_1_tst, columns=['final_1_tst']), pd.DataFrame(final_2_tst,columns=['final_2_tst']),\
           pd.DataFrame(final_3_tst,columns=['final_3_tst']),pd.DataFrame(final_4_tst,columns=['final_4_tst']),\
           pd.DataFrame(final_5_tst,columns=['final_5_tst'])],axis=1)

final_tst_preds.head(2)

,final_1_tst,final_2_tst,final_3_tst,final_4_tst,final_5_tst
0,0.622927,0.622306,0.583008,0.499456,0.612489
1,0.381731,0.368453,0.335205,0.443994,0.407913


In [63]:
pred_val_array = meta_model.predict(final_val_preds)
pred_test_array = meta_model.predict(final_tst_preds)

pred_val_array = np.reshape(pred_val_array, (-1, 28),order = 'F')
pred_test_array = np.reshape(pred_test_array, (-1, 28),order = 'F')

In [82]:
final_val_preds_cbr = final_val_preds
final_val_preds_cbr.columns =  ['k1_pred','k2_pred','k3_pred','k4_pred','k5_pred']
final_val_preds_cbr


final_tst_preds_cbr = final_tst_preds
final_tst_preds_cbr.columns =  ['k1_pred','k2_pred','k3_pred','k4_pred','k5_pred']
final_tst_preds_cbr

,k1_pred,k2_pred,k3_pred,k4_pred,k5_pred
0,0.622927,0.622306,0.583008,0.499456,0.612489
1,0.381731,0.368453,0.335205,0.443994,0.407913
2,0.547043,0.552569,0.509033,0.493762,0.568509
3,2.417879,2.438912,2.388916,2.057252,2.078961
4,1.336990,1.320781,1.288330,0.870764,1.065992
...,...,...,...,...,...
853715,0.949158,0.975781,1.009766,0.761162,0.862908
853716,0.346672,0.321799,0.387695,0.440660,0.439751
853717,1.184471,1.172040,1.246582,0.785601,0.890500
853718,1.110918,1.073043,1.143311,0.910342,1.025053


In [87]:
pred_val_array = meta_model_cbr.predict(final_val_preds)
pred_test_array = meta_model_cbr.predict(final_tst_preds)

pred_val_array = np.reshape(pred_val_array, (-1, 28),order = 'F')
pred_test_array = np.reshape(pred_test_array, (-1, 28),order = 'F')

In [88]:
sub = pd.read_csv("sample_submission.csv")
sub_1 = sub.iloc[:30490,:]
sub_2 = sub.iloc[30490:,:]
f_cols = sub.columns[1:]

for i in range(len(f_cols)):
    sub_1[f_cols[i]] = pred_val_array[:,i]
    sub_2[f_cols[i]] = pred_test_array[:,i]

sub = pd.concat([sub_1,sub_2])
sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.814221,0.712517,0.710298,0.445180,0.967422,0.730161,0.872007,0.714305,1.316507,0.812537,1.490221,1.129385,1.130636,1.115721,0.491443,1.231129,0.714676,0.829558,0.859721,1.086065,0.965488,0.809171,0.913963,0.815256,1.290691,0.855042,1.257858,0.855042
1,HOBBIES_1_002_CA_1_validation,0.552600,0.319139,0.295450,0.495463,0.297214,0.399414,0.372379,0.400019,0.240263,0.210431,0.497141,0.316421,0.327907,0.334915,0.337549,0.213935,0.209037,0.420556,0.333424,0.354352,0.354352,0.338684,0.206539,0.293567,0.335687,0.357595,0.344008,0.292112
2,HOBBIES_1_003_CA_1_validation,0.393949,0.373498,0.244875,0.213535,0.472361,0.910746,0.565883,0.334095,0.547115,0.216592,0.374260,0.385460,1.009061,0.481869,0.253814,0.354638,0.374260,0.660056,0.794580,1.152470,1.033813,0.570897,0.804832,0.605386,0.606789,0.798736,0.648553,0.820394
3,HOBBIES_1_004_CA_1_validation,2.177026,1.033804,0.712517,1.442463,1.429108,1.790227,3.127834,1.521750,1.147287,0.806250,1.496973,1.560228,2.006013,2.925439,2.006013,1.243791,1.437747,0.919120,1.601964,1.790227,2.924025,1.999796,1.378379,1.473003,1.163040,1.683155,2.919750,2.679918
4,HOBBIES_1_005_CA_1_validation,1.115721,0.955339,1.247951,1.162885,1.332635,1.147287,1.445988,0.971916,1.360630,0.952926,1.258334,0.733018,0.919586,1.115721,1.190321,0.959141,1.186222,0.870851,1.129385,0.934649,1.310323,0.980431,0.925658,0.916693,1.115721,1.337398,1.183100,1.865430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.438759,0.368562,0.262954,0.712517,0.730161,0.505131,0.639616,0.397608,0.571548,0.287307,0.768650,0.803426,0.457681,0.660456,0.552128,0.561156,0.430496,0.525211,0.955676,0.372833,1.090080,0.392350,1.047022,0.536635,0.559752,0.828874,0.524276,0.814088
60976,FOODS_3_824_WI_3_evaluation,0.564157,0.519263,0.392350,0.524276,0.352822,0.577073,0.385460,0.433129,0.556547,0.372836,0.724753,0.238851,0.662668,0.366257,0.523358,0.457681,0.336789,0.561796,0.217933,0.501757,0.348675,0.495707,0.471218,0.342167,0.657387,0.203985,0.656635,0.299036
60977,FOODS_3_825_WI_3_evaluation,1.258334,0.529683,0.805452,0.392743,0.726573,1.033813,0.661916,1.356637,0.355845,0.809171,0.465945,0.557391,0.925658,0.829532,1.103459,0.912774,1.270019,0.959691,0.714305,0.921802,0.798792,1.159081,0.717162,1.382818,0.433881,0.569319,0.609005,1.033813
60978,FOODS_3_826_WI_3_evaluation,0.932127,1.743731,0.714305,1.351129,1.131112,0.959141,1.517481,1.131112,1.031689,1.041049,1.131112,1.036466,1.271389,1.457660,1.409498,0.919784,1.011393,1.115721,1.229889,1.086065,1.139583,1.441719,1.996701,1.324901,1.777124,1.015408,1.090080,1.035215


In [90]:
sub.to_csv("stack_model_3.csv",index = False)

In [91]:
!kaggle competitions submit -c m5-forecasting-accuracy -f stack_model_3.csv -m "Third stacked model"

100% 32.7M/32.7M [00:04<00:00, 6.95MB/s]
Successfully submitted to M5 Forecasting - Accuracy